# Preparing model architecture

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="attention_weights",
                                 shape=(input_shape[-1], 1),
                                 initializer="glorot_uniform",
                                 trainable=True)
        super().build(input_shape)

    def call(self, inputs):
        # Compute attention scores
        e = tf.matmul(inputs, self.W)
        alpha = tf.nn.softmax(e, axis=1)
        context = tf.reduce_sum(alpha * inputs, axis=1)
        return context, tf.squeeze(alpha, -1)  # output attention weights too

def build_attention_model(input_dim, num_classes, return_attention=False):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = tf.expand_dims(inputs, axis=1)

    attention_layer = AttentionLayer()
    attention_output, attention_weights = attention_layer(x)

    x = tf.keras.layers.Dense(64, activation='relu')(attention_output)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    if return_attention:
        model = tf.keras.Model(inputs=inputs, outputs=[output, attention_weights])
    else:
        model = tf.keras.Model(inputs=inputs, outputs=output)

    return model

# Creating Entity method for model directory

In [21]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelPreparationConfig:
    root_dir: Path
    training_independent_data_path: Path
    training_dependent_data_path: Path
    model_saved_path: Path
    epochs: int
    batch_size: int
    optimizer: str
    loss: str
    metrics: str

# Preparing Configuration Manager

In [6]:
import os
os.chdir('../')

In [7]:
from src.No_More_Lapses.constants import *
from src.No_More_Lapses.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_preparation_config(self) -> ModelPreparationConfig:
        config = self.config.model_trainer
        params = self.params.training_hyperparameters

        create_directories([config.root_dir])
        
        data_transformation_config = ModelPreparationConfig(
            root_dir=config.root_dir,
            training_independent_data_path = config.training_independent_data_path,
            training_dependent_data_path = config.training_dependent_data_path,
            model_saved_path = config.model_saved_path,
            epochs=params.EPOCHS,
            batch_size=params.BATCH_SIZE,
            optimizer=params.OPTIMIZER,
            loss=params.LOSS,
            metrics=params.METRICS
        )

        return data_transformation_config

### Testing the configuration manager

In [23]:
a = ConfigurationManager()

[2025-03-24 11:51:43,916: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-24 11:51:43,919: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-24 11:51:43,920: INFO: common: created directory at: artifacts]


In [24]:
a.get_model_preparation_config()

[2025-03-24 11:51:56,520: INFO: common: created directory at: artifacts/model_trainer]


ModelPreparationConfig(root_dir='artifacts/model_trainer', training_independent_data_path='artifacts/transformed_data/trainining_data/X_train.csv', training_dependent_data_path='artifacts/transformed_data/trainining_data/y_train.csv', model_saved_path='artifacts/model_trainer/attention_model.h5', epochs=20, batch_size=32, optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

# Model Preparation

In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from src.No_More_Lapses import logger

In [53]:
class ModelPreparation:
    
    def __init__(self, config: ModelPreparationConfig):
        self.config = config

    def trainModel(self):
        X_train = pd.read_csv(self.config.training_independent_data_path)
        y_train = pd.read_csv(self.config.training_dependent_data_path)
        logger.info("Train data loaded")

        X_train = X_train.drop(columns=['Unnamed: 0'],errors='ignore')
        y_train = y_train.drop(columns=['Unnamed: 0'],errors='ignore')
        logger.info("Removing the index column")

        y_series = y_train["POLICY STATUS"]  # explicitly get the column
        num_classes = len(y_series.unique())

        # Step 4: Convert to categorical
        y_encoded = tf.keras.utils.to_categorical(y_series, num_classes)
        logger.info("Converted y to one-hot encoding")
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)
        logger.info("Applied Standard Scaler to scale down the values in the same range to avoid gradient boosting or biases")

        model = build_attention_model(input_dim=X_scaled.shape[1], num_classes=num_classes, return_attention=False)


        model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
            )
        
        model.fit(X_scaled, y_encoded, epochs=self.config.epochs, batch_size=self.config.batch_size)
        attention_model = build_attention_model(input_dim=X_scaled.shape[1], num_classes=num_classes, return_attention=True)
        attention_model.set_weights(model.get_weights())
        # Save model and scaler
        model.save(self.config.model_saved_path)
        return model, scaler




### Training the model (test version)

In [54]:
model_caller = ModelPreparation(config=a.get_model_preparation_config())

[2025-03-24 12:15:28,357: INFO: common: created directory at: artifacts/model_trainer]


In [55]:
model_caller.trainModel()

[2025-03-24 12:15:29,898: INFO: 2486981150: Train data loaded]
[2025-03-24 12:15:29,917: INFO: 2486981150: Removing the index column]
[2025-03-24 12:15:29,925: INFO: 2486981150: Converted y to one-hot encoding]
[2025-03-24 12:15:29,973: INFO: 2486981150: Applied Standard Scaler to scale down the values in the same range to avoid gradient boosting or biases]
Epoch 1/20


2025-03-24 12:15:30.311074: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12324/12324 [==============================] - 225s 18ms/step - loss: 0.8534 - accuracy: 0.6328
Epoch 2/20
12324/12324 [==============================] - 222s 18ms/step - loss: 0.7677 - accuracy: 0.6680
Epoch 3/20
12324/12324 [==============================] - 223s 18ms/step - loss: 0.7387 - accuracy: 0.6804
Epoch 4/20
12324/12324 [==============================] - 224s 18ms/step - loss: 0.7195 - accuracy: 0.6880
Epoch 5/20
12324/12324 [==============================] - 225s 18ms/step - loss: 0.7058 - accuracy: 0.6931
Epoch 6/20
12324/12324 [==============================] - 225s 18ms/step - loss: 0.6962 - accuracy: 0.6967
Epoch 7/20
12324/12324 [==============================] - 224s 18ms/step - loss: 0.6878 - accuracy: 0.6997
Epoch 8/20
12324/12324 [==============================] - 225s 18ms/step - loss: 0.6819 - accuracy: 0.7023
Epoch 9/20
12324/12324 [==============================] - 225s 18ms/step - loss: 0.6769 - accuracy: 0.7043
Epoch 10/20
12324/12324 [=======================

(<keras.engine.functional.Functional at 0x2cc1eabe0>, StandardScaler())

# Baseline Model

In [56]:
training_independent_data_path='artifacts/transformed_data/trainining_data/X_train.csv'
training_dependent_data_path='artifacts/transformed_data/trainining_data/y_train.csv'

In [57]:
X = pd.read_csv(training_independent_data_path)
y = pd.read_csv(training_dependent_data_path)

X_train = X.drop(columns=['Unnamed: 0'],errors='ignore')
y_train = y.drop(columns=['Unnamed: 0'],errors='ignore')

y_series = y_train["POLICY STATUS"]  # explicitly get the column
num_classes = len(y_series.unique())

# Step 4: Convert to categorical
y_encoded = tf.keras.utils.to_categorical(y_series, num_classes)
logger.info("Converted y to one-hot encoding")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

[2025-03-24 14:15:31,894: INFO: 3808260860: Converted y to one-hot encoding]


In [61]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 56.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 60.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 55.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.0/681.0 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import mlflow
import joblib

In [69]:
mlflow.set_tracking_uri("file:mlruns")

clf = RandomForestClassifier()
clf.fit(X_scaled, y_series)
y_pred = clf.predict(X_scaled)
acc = accuracy_score(y_series, y_pred)
report=classification_report(y_series, y_pred)
joblib.dump(clf, 'artifacts/model_trainer/baseline_model.h5')

with open("logs/rf_report.txt", "w") as f:
    f.write(report)

with open("logs/rf_report.txt", "w") as f:
    f.write(report)

with mlflow.start_run(run_name="baseline_RandomForestClassifier"):
    mlflow.log_param("model_type", "RandomForestClassifier")
    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(clf, "model", registered_model_name="baseline_RandomForestClassifier")
    mlflow.log_artifact('artifacts/model_trainer')
    mlflow.log_artifact("artifacts/model_trainer/rf_report.txt")



2025/03/24 14:58:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'baseline_RandomForestClassifier'.
Created version '1' of model 'baseline_RandomForestClassifier'.
